In [1]:
import pandas as pd
import matplotlib.pyplot as plt 

In [2]:
import urllib
params ='DRIVER={SQL Server};SERVER=CL-AMILAH\AMILA;DATABASE=GSH;UID=Amila;Trusted_Connection=yes;'             
params = urllib.parse.quote_plus(params)

from sqlalchemy import create_engine
db = create_engine('mssql+pyodbc:///?odbc_connect=%s' % params)

# Load sales data

In [3]:
sql = '''
SELECT inv.[ID] ,inv.SiteUID, [Date] , [OutletUID] ,ot.Name AS outlet ,ot.ProvinceUID,pr.Description AS province ,ot.DistrictUID ,  di.Description AS district 
, ot.AreaUID , ar.Description AS Area , inv.[AgentUID] ,Concat(emp.FirstName ,' ' , emp.MiddleInitials , ' ', emp.LastName ) AS AgentName , inv.[DistributorUID] , d.Name AS distributor, [ItemUID] ,it.Description AS item, [UnitQty] 
, [UnitPrice] , [RetailUnitPrice] , [FreeQty] , [FreeCompanyQty] , [GrossValue]
,datename(YEAR, Date) AS Year
,datename(month, Date) AS MonthName
,MONTH(Date) AS Month
,datename(DAY, Date) AS Day
,DATEPART(DW,Date) AS WeekDay
,datename(WEEKDAY, Date) AS DateName
,DATEPART (hh,Date) as Hour

  FROM [GSH].[dbo].[vw_Invoice] inv
  LEFT JOIN [GSH].[dbo].[vw_InvoiceLine] invl
  ON inv.siteUID = invl.TxnInvoice_SiteUID AND inv.uid = invl.TxnInvoiceUID	
  LEFT JOIN [GSH].[dbo].[Item] it
  ON invl.ItemUID = it.UID
  LEFT JOIN [GSH].[dbo].[Outlet] ot
  ON inv.OutletUID = ot.UID
  LEFT JOIN [GSH].[dbo].[Area] pr
  ON ot.ProvinceUID = pr.UID
  LEFT JOIN [GSH].[dbo].[Area] di
  ON ot.DistrictUID = di.UID
  LEFT JOIN [GSH].[dbo].[Area] ar
  ON ot.AreaUID = ar.UID
  LEFT JOIN [GSH].[dbo].[Distributor] d
  ON inv.DistributorUID = d.UID
  LEFT JOIN [GSH].[dbo].[avLogin] avlg
  ON inv.AgentUID = avlg.BusinessUserUID
  LEFT JOIN [GSH].[dbo].[Employee] emp
  ON avlg.EmployeeUID = emp.UID
  WHERE inv.mpt_TypeEnum IN ( 21 , 44, 14, 1) AND inv.ExpiryDate IS NULL AND inv.SubmittedDate IS NOT NULL
  AND Date >= '2018-11-01' AND Date <= '2020-03-01' ;
'''

invoice_All = pd.read_sql_query(sql, db)

In [4]:
#disply all the columns
pd.set_option('display.max_columns', None)

In [5]:
invoice_All.head()

,ID,SiteUID,Date,OutletUID,outlet,ProvinceUID,province,DistrictUID,district,AreaUID,Area,AgentUID,AgentName,DistributorUID,distributor,ItemUID,item,UnitQty,UnitPrice,RetailUnitPrice,FreeQty,FreeCompanyQty,GrossValue,Year,MonthName,Month,Day,WeekDay,DateName,Hour
0,CSO007901614,53,2018-11-07 08:04:10.913,389,UNION PHARMACY,24.0,North Western Province,26.0,Puttalam,92.0,CHILAW,79,Shashikala perera,2,The West Central Pharma (Pvt) Ltd.,25,Aluminum hydroxide 240ml,10.0,238.7,272.0,0.0,0.0,2387.0,2018,November,11,7,4,Wednesday,8
1,CSO007901614,53,2018-11-07 08:04:10.913,389,UNION PHARMACY,24.0,North Western Province,26.0,Puttalam,92.0,CHILAW,79,Shashikala perera,2,The West Central Pharma (Pvt) Ltd.,25,Aluminum hydroxide 240ml,0.0,238.7,272.0,3.0,3.0,0.0,2018,November,11,7,4,Wednesday,8
2,CSO007901614,53,2018-11-07 08:04:10.913,389,UNION PHARMACY,24.0,North Western Province,26.0,Puttalam,92.0,CHILAW,79,Shashikala perera,2,The West Central Pharma (Pvt) Ltd.,106,Artisol - Tablets (2x15's),1.0,1309.3,1492.5,0.0,0.0,1309.3,2018,November,11,7,4,Wednesday,8
3,CSO009400331,107,2018-11-07 10:38:10.487,932,Care Pharmacy,20.0,Western Province,22.0,Kalutara,145.0,Horana,94,Sumedha Kulathunga,9,Multy Drugs (Pvt) Ltd,25,Aluminum hydroxide 240ml,50.0,238.7,272.0,0.0,0.0,11935.0,2018,November,11,7,4,Wednesday,10
4,CSO011701466,54,2018-11-14 10:21:37.800,388,SUHADA PHARMACY,24.0,North Western Province,26.0,Puttalam,92.0,CHILAW,117,,2,The West Central Pharma (Pvt) Ltd.,97,Peridex and PerioGard 15mg,5.0,157.1,179.0,0.0,0.0,785.5,2018,November,11,14,4,Wednesday,10


### What products are most often sold together?

In [6]:
Sold_Together =invoice_All.groupby(["ID" , "item"]).agg({'GrossValue':'sum'}).reset_index()

In [7]:
Sold_Together = Sold_Together[Sold_Together['ID'].duplicated(keep=False)]

Sold_Together['Grouped'] = Sold_Together.groupby('ID')['item'].transform(lambda x: ','.join(x))
Sold_Together_final = Sold_Together[['ID', 'Grouped']].drop_duplicates()

In [8]:
from itertools import combinations
from collections import Counter

count = Counter()

for row in Sold_Together_final['Grouped']:
    row_list = row.split(',')
    count.update(Counter(combinations(row_list, 2)))

for key,value in count.most_common(10):
    print(key, value)

('Cilnidipine tab 10mg ', 'Cilnidipine tab 5mg') 8892
('nitroimidazoles GEL 20mg', 'nitroimidazoles GEL 30mg') 8091
('Clobetasol G cream 15mg', 'Gentamicin\xa0Cream 15 mg') 7998
('Nifedipine 30mg', 'nitroimidazoles GEL 30mg') 7884
('Clobetasol G cream 15mg', 'benzoyl peroxide wash 60mg') 7279
('Cilnidipine tab 5mg', 'nitroimidazoles GEL 30mg') 7197
('Cilnidipine tab 5mg', 'Nifedipine 30mg') 6072
('hypoglycaemic sulfonylurea antidiabetic 80mg', 'nitroimidazoles GEL 30mg') 5733
('Clobetasol G cream 15mg', 'Gentamicin 15 mg') 5723
('Benzoyl Peroxide', 'Clobetasol G cream 15mg') 5450


In [9]:
print(Sold_Together_final['ID'].value_counts())

CSO018400452    1
CSO018800015    1
CSO008102974    1
CSO008602002    1
CSO009203543    1
               ..
CSO008200790    1
CSO008203637    1
CSO011703705    1
CSO015603750    1
CSO010404509    1
Name: ID, Length: 135529, dtype: int64
